# Activations, Gradients, BatchNorm

Formerly:

- Used MLP to predict the next character in the sequence

Now:

- A little more about MLPs: activations and gradients
- "Universal approximator" but not easily optimizable because of activations and gradients
- Newer approaches attempt to handle activations and gradients better 

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random

%matplotlib inline

import matplotlib

Read in all the names

In [ ]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(words[:8])

Build the vocabulary of characters and mappings to/from integers

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)

print(itos)
print(vocab_size)

## Build the dataset

In [ ]:
block_size = 3

def build_dataset(words):
  X, Y = [], []
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

## Set up general MLP

In [ ]:
n_embed = 10    # dimensionality of the character embedding vectors
n_hidden = 200  # number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator = g)
W1 = torch.randn((n_embed * block_size, n_hidden), generator = g) * (5/3) / ((n_embed * block_size)**0.5)
# b1 = torch.randn(n_hidden, generator = g) * 0.01
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.01 # Scale down but not 0
b2 = torch.randn(vocab_size, generator = g) * 0 # Initialize at 0

# Scale and shift
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(f'Network size: {sum(p.nelement() for p in parameters)}')

for p in parameters:
  p.requires_grad = True

## Optimization

In [ ]:
max_steps = 200000
batch_size = 32
loss_ii = []

for ii in range(max_steps):
  # Minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator = g)
  Xb, Yb = Xtr[ix], Ytr[ix]  # Batches X and Y

  # Forward pass
  emb = C[Xb] # Embed characters into vectors
  embcat = emb.view(emb.shape[0], -1) # Concatenate the vectors

  # Linear layer
  # Note no b1 here, because of standardization below. All these
  # gradients are 0. bnbias takes care of it.
  hpreact = embcat @ W1 # Hidden layer preactivation

  # Batch normalization after linear layer
  # Add a small amount to std_ii to avoid divide by zero
  bnmean_ii = hpreact.mean(0, keepdim = True)
  bnstd_ii = hpreact.std(0, keepdim = True)
  hpreact = bngain * (hpreact - bnmean_ii) / (bnstd_ii + 1e-5) + bnbias

  # Update running mean and sd (don't keep gradients)
  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 * bnmean_ii
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstd_ii

  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb)

  # Backward pass
  for p in parameters:
    p.grad = None # initialize to 0
  loss.backward()

  # Update
  lr = 0.1 if ii < 100000 else 0.01
  for p in parameters:
      p.data += -lr * p.grad
  
  # Print
  if ii % 10000 == 0:
    print(f'{ii:7d} / {max_steps:7d}: {loss.item():.4f}')

  # Keep track of stats
  loss_ii.append(loss.log10().item())

  # if ii > 1000:
  #   break

print(loss.item())

The initialization is not very good (27 is really high)

- The distribution should be uniform at the beginning


In [ ]:
-torch.tensor(1 / 27.0).log()

Should be ~3.3

Scale down W2 and b2 at initialization.

- Set b2 to 0
- Scale W2 to small numbers to tame the loss at initialization
- Spend more time optimizing the NN

At initialization, `tanh` is pushing too many values to -1 and 1:

- Backpropagation through `tanh` with -1 or 1 stops the backpropagation.
- Changing the input doesn't change the output (no impact on loss)
- Gradient is squashed
- When `tanh` = 0, `out.grad` is passed through
- `hpreact` is too broad (-15 to 15) at initialization
- Scale down `b1` and `W1`

"Dead neurons" happen when all the gradients are zeroed for a single neuron.

- Can happen for tanh, relu, sigmoid or any distribution with a flat region
- Other activation functions have been developed to avoid this flatness

In [ ]:
plt.hist(h.view(-1).tolist(), 50);

In [ ]:
plt.hist(hpreact.view(-1).tolist(), 50);

White are saturated neurons

In [ ]:
plt.figure(figsize=(20, 15))
plt.imshow(h.abs() > 0.99, cmap = 'gray', interpolation = 'nearest');

Plot loss


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(loss_ii);

Compare train and validate


In [ ]:
@torch.no_grad() # Decorator to disable gradient tracking in the following
def split_loss(split):
  x, y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split]

  # Forward pass
  emb = C[x]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 # Hidden layer preactivation

  # Can't calculate the local batch normalization for single samples, so
  # use the running values from above.
  # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim = True)) / hpreact.std(0, keepdim = True) + bnbias

  # Use the calculated batch normalization from above
  hpreact = bngain * (hpreact - bnmean_running) / bnstd_running + bnbias

  h = torch.tanh(hpreact)
  logits = h @ W2 + b2

  # Evaluate loss
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

Sample from the model


In [ ]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
  out = []
  context = [0] * block_size

  while True:
    emb = C[torch.tensor([context])]
    h = torch.tanh(emb.view(1, -1) @ W1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim = 1)

    # Sample from the multinomial distribution
    ix = torch.multinomial(probs, num_samples = 1, generator = g).item()

    # Shift the context window and track the samples
    context = context[1:] + [ix]
    out.append(ix)

    # Break if we hit '.'
    if ix == 0:
      break
  print(''.join(itos[i] for i in out))


The network is shallow and simple, so the optimization eventually works fine, even with the poor initialization. This will not be the case with much deeper networks (e.g., 50 layers).

- Bad initialization can mean that the network will not trait at all.

How do you determine what the values are for scaling the initialization (0.2, 0.01, etc. above)?

Consider matrix multiplication of random normal values:


In [ ]:
x = torch.randn(1000, 10)
w = torch.randn(10, 200)
y = x @ w
print(x.mean(), x.std())
print(y.mean(), y.std())

plt.figure(figsize = (20, 5))
plt.subplot(121)
plt.hist(x.view(-1), 50, density = True);
plt.subplot(122)
plt.hist(y.view(-1), 50, density = True);

Mean stays at 0, sd goes from 1 to 3.

How can we preserve the standard normal?

- Divide by sqrt of "fan_in" (here, 10)
- See He et al., 2015 about CNN and ReLU / PReLu
- If the forward pass is optimized, the backward pass will be as well
- See `torch.nn.kaiming_normal` for standard initialization built-in to pytorch
    - Has proper gain for the activation function
- Modern innovations have made the initialization less crucial: better optimizers, other activations, batch normalization


In [ ]:
x = torch.randn(1000, 10)
w = torch.randn(10, 200) / 10**0.5
y = x @ w
print(x.mean(), x.std())
print(y.mean(), y.std())

plt.figure(figsize = (20, 5))
plt.subplot(121)
plt.hist(x.view(-1), 50, density = True);
plt.subplot(122)
plt.hist(y.view(-1), 50, density = True);

Batch Normalization

- Don't preactivation states to be too small or too large.
- Just standardize the hidden states to make them standard Normal
- Work on `hpreact`
- Only want it to be standardized at initialization. Want it to be able to move around in backpropagation.
    - Scale and shift as a final step.

We still don't get much improvement from batch normalization because the network is pretty small.

- Scatter batch normalization around the network. One per linear or convolutional layer
- There is a cost: neurons are not independent anymore, depending on what else is in the batch will change `h` and `logits`.
- This acts as a regularizer via (a kind of) data augmentation

Batch normalization causes difficulties for new observation prediction, because it expects to calculate mean and standard deviation for single observations.

- One solution is to have a post-training step that fixes the batch normalization one time.
- Better solution is to do the updating during training, updating a little at each iteration (implemented above)

Group and layer normalization are newer versions of batch normalization.

## Implementation in torch


In [ ]:
class Linear:

  def __init__(self, fan_in, fan_out, bias = True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])


class BatchNorm1d:

  def __init__(self, dim, eps = 1e-5, momentum = 0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True

    # Parameters trained (with backpropagation)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

    # Buffers (trained with running momentum update)
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
    
  def __call__(self, x):

    # Calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdims = True) # Batch mean
      xvar = x.var(0, keepdims = True, unbiased = True) # Batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
    self.out = self.gamma * xhat + self.beta

    # Update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]


class Tanh:

  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  
  def parameters(self):
    return []

n_embd = 10
n_hidden = 100
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator = g)
layers = [
  Linear(n_embd * block_size, n_hidden), Tanh(),
  Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size)
]

with torch.no_grad():
  # Last layer: make less confident
  # layers[-1].weight += 0.1
  layers[-1].gamma += 0.1


  # All other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]

print(f'Number of parameters: {sum(p.nelement() for p in parameters)}')
for p in parameters:
  p.requires_grad = True


## Optimization

In [ ]:
max_steps = 200000
batch_size = 32
loss_ii = []
ud = []

for ii in range(max_steps):

  # Minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator = g)
  Xb, Yb = Xtr[ix], Ytr[ix]

  # Forward pass
  emb = C[Xb] # Embed characters into a vector
  x = emb.view(emb.shape[0], -1) # Concatenate the vectors
  for layer in layers:
      x = layer(x)
  loss = F.cross_entropy(x, Yb)

  # Backward pass
  for layer in layers:
    layer.out.retain_grad()
  for p in parameters:
    p.grad = None
  loss.backward()

  # Update
  lr = 0.1 if ii < 10000 else 0.01
  for p in parameters:
      p.data += -lr * p.grad
  
  # Track stats
  if ii % 10000 == 0:
    print(f'{ii:7d} / {max_steps:7d}: {loss.item():.4f}')
  loss_ii.append(loss.log10().item())

  # Data update ratio: how large are the changes to data relative to gradient
  with torch.no_grad():
    ud.append([(lr * p.grad.std() / p.data.std()).log10().item() for p in parameters])

  # if ii > 1000:
  #   break


## Visualize activation distribution


In [ ]:
plt.figure(figsize = (20, 4))
legends = []

for i, layer in enumerate(layers[:-1]):  # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated %.2f%%' % 
    (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density = True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends);
plt.title('activation distribution')